<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/43_extract_values_to_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# FULL SCRIPT
import os
import ee
import geemap
import geopandas

# Define Variables
id_attributte = 'TarlaId';
polygon_name = ee.String('users/batuhankavlak4/{YOUR_POLYGON_ASSET}');
ee_dataset = "COPERNICUS/S2_SR";
date_start = ee.Date('2020-02-01');
date_end = ee.Date('2020-11-30');
cloud_perc = 20;
band_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12'];


def set_field_id(id_int):
    '''
    Adds integer as FieldId att to an EE feature object.

    Parameters:
        feature: an EE feature object.
        id_int (int): Id to be added.

    Returns:
        feature(ee feature): an EE feature object. 
    '''
    def wrapper_f(feature):
        base_dict = ee.Dictionary({
            'FieldId': id_int
        })
        feature = feature.set(base_dict)
        return feature
    return wrapper_f


def set_coord_att(feature):
    '''
    Adds coordinates as Coords att to an EE feature object.

    Parameters:
        feature: an EE feature object.

    Returns:
        feature(ee feature): an EE feature object. 
    '''
    coords = feature.geometry().coordinates()
    base_dict = ee.Dictionary({
        'Coords': coords
    })
    feature = feature.set(base_dict)
    return feature


# Generic Function to remove a property from a feature
def remove_property(property_name):
    '''
    Remove only one att from an EE feature object.

    Parameters:
        feature: an EE feature object.
        property_name (str): Attribute to be dropped.

    Returns:
        feature(ee feature): an EE feature object. 
    '''
    def wrapper_f(feature):
        properties = feature.propertyNames()
        select_properties = properties.filter(ee.Filter.neq('item', property_name))
        return feature.select(select_properties)
    return wrapper_f
        
    

# Get Target Polygon File
polygon_collection = ee.FeatureCollection(polygon_name);
# Convert  in a Python list
polygon_collection_list = polygon_collection.aggregate_array(id_attributte);
polygon_collection_list_py = polygon_collection_list.getInfo();

# LOOP 1 - OVER POLYGONS
for field_id in polygon_collection_list_py:
    
    # Get target polygon
    polygon_feature = ee.Feature(polygon_collection.filter(ee.Filter.eq(id_attributte, field_id)).first());
    
    # Add Earth Engine dataset
    s2_col = ee.ImageCollection(ee_dataset) \
    .filterBounds(polygon_feature.geometry()) \
    .filterDate(date_start, date_end) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_perc)) \
    .select(band_names);
    
    # Extract Pixels as Point
    base_image = ee.Image(ee.ImageCollection("COPERNICUS/S2_SR") \
                               .filterBounds(polygon_feature.geometry()) \
                               .filterDate(date_start, date_end) \
                               .first() \
                               .select(['B2']))
    polygon_feature =  base_image \
    .sample(**{
        'region': polygon_feature.geometry(),
        'geometries': True,  # if you want points,
        'scale': 10,
        'projection': base_image.projection().crs()
    });
    
    # Add FieldId & Coords
    polygon_feature = polygon_feature.map(set_field_id(field_id))
    polygon_feature = polygon_feature.map(set_coord_att)
    # Remove B2 & system:index
    polygon_feature = polygon_feature.map(remove_property('B2'))
    polygon_feature = polygon_feature.map(remove_property('system:index'))
    
    # Get PRODUCT_ID in a Python list
    collection_list = s2_col.aggregate_array('PRODUCT_ID');
    collection_list_py = collection_list.getInfo();
    
    # LOOP 2 - OVER IMAGES
    for product_name in collection_list_py:
        new_band_names = [product_name + '_' + s for s in band_names]
        # Add Earth Engine dataset
        s2_image = ee.Image(s2_col \
                            .filter(ee.Filter.eq('PRODUCT_ID', product_name)) \
                            .first()) \
        .rename(new_band_names);
        output_dir = os.path.expanduser('./csv/');
        out_csv = os.path.join(output_dir, product_name + '_' + str(field_id) + '.csv');
        geemap.extract_values_to_points(polygon_feature, s2_image, out_csv, scale = 10);
    

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200203T083049_N0213_R021_T36SVG_20200203T115303_325042.csv
Generating URL ...
Please wait ...
An error occurred while downloading. 
 Retrying ...
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200218T083011_N0214_R021_T36SVG_20200218T104655_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200226T083829_N0214_R064_T36SVG_20200226T134510_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200302T083841_N0214_R064_T36SVG_20200302T120522_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200307T083719_N0214_R064_T36SVG_20200307T131137_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/tem

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200630T083611_N0214_R064_T36SVG_20200630T115950_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200702T082609_N0214_R021_T36SVG_20200702T122946_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200705T083559_N0214_R064_T36SVG_20200705T131340_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200707T082611_N0214_R021_T36SVG_20200707T115029_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200710T083601_N0214_R064_T36SVG_20200710T112552_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200712T082609_N0214_R021_T36SVG_20200712T130343_325

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200826T082611_N0214_R021_T36SVG_20200826T114808_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200829T083611_N0214_R064_T36SVG_20200829T112650_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200831T082609_N0214_R021_T36SVG_20200831T123531_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200903T083609_N0214_R064_T36SVG_20200903T123859_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200908T083601_N0214_R064_T36SVG_20200908T103544_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200910T082609_N0214_R021_T36SVG_20200910T111619_325

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20201112T084209_N0214_R064_T36SVG_20201112T111055_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20201114T083221_N0214_R021_T36SVG_20201114T112303_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20201117T084241_N0214_R064_T36SVG_20201117T110851_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20201124T083301_N0214_R021_T36SVG_20201124T114704_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20201127T084311_N0214_R064_T36SVG_20201127T115456_325042.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20201129T083309_N0214_R021_T36SVG_20201129T111818_325

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200607T082611_N0214_R021_T36SVG_20200607T115845_325043.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200610T083611_N0214_R064_T36SVG_20200610T112436_325043.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200612T082609_N0214_R021_T36SVG_20200612T105411_325043.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200615T083559_N0214_R064_T36SVG_20200615T123803_325043.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200622T082609_N0214_R021_T36SVG_20200622T123135_325043.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200627T082611_N0214_R021_T36SVG_20200627T113148_325

...

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20201124T083301_N0214_R021_T36SWG_20201124T114704_328495.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20201127T084311_N0214_R064_T36SWG_20201127T115456_328495.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20201129T083309_N0214_R021_T36SWG_20201129T111818_328495.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200218T083011_N0214_R021_T36SWG_20200218T104655_328500.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200413T082559_N0214_R021_T36SWG_20200413T115830_328500.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200428T082601_N0214_R021_T36SWG_20200428T113331_328

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200826T082611_N0214_R021_T36SWG_20200826T114808_328500.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200831T082609_N0214_R021_T36SWG_20200831T123531_328500.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200910T082609_N0214_R021_T36SWG_20200910T111619_328500.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200915T082611_N0214_R021_T36SWG_20200915T105159_328500.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200920T082629_N0214_R021_T36SWG_20200920T112156_328500.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200930T082739_N0214_R021_T36SWG_20200930T111821_328

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200426T083559_N0214_R064_T36SVG_20200426T123506_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200428T082601_N0214_R021_T36SVG_20200428T113331_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200506T083559_N0214_R064_T36SVG_20200506T131335_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200513T082559_N0214_R021_T36SVG_20200513T115741_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200516T083559_N0214_R064_T36SVG_20200516T123826_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200518T082611_N0214_R021_T36SVG_20200518T111906_324

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200722T082609_N0214_R021_T36SVG_20200722T121557_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200725T083559_N0214_R064_T36SVG_20200725T123802_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200727T082611_N0214_R021_T36SVG_20200727T112047_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200730T083611_N0214_R064_T36SVG_20200730T114836_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200801T082609_N0214_R021_T36SVG_20200801T122836_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200804T083609_N0214_R064_T36SVG_20200804T131705_324

Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200923T083659_N0214_R064_T36SVG_20200923T124210_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200925T082711_N0214_R021_T36SVG_20200925T122704_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20200928T083741_N0214_R064_T36SVG_20200928T115710_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20200930T082739_N0214_R021_T36SVG_20200930T111821_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2B_MSIL2A_20201003T083729_N0214_R064_T36SVG_20201003T120429_324732.csv
Generating URL ...
Please wait ...
Data downloaded to /geemap/data/PC/temp/earth-extraction/csv/S2A_MSIL2A_20201005T082821_N0214_R021_T36SVG_20201005T144124_324

In [1]:
import os
import ee
import geemap
import geopandas

In [2]:
Map = geemap.Map()
Map

Enter verification code: 4/1AX4XfWjVRvPleR7F1C3DPzzkXMB6kuUvEiDLiA3mkUjqUV_voo_qeKLDVoM

Successfully saved authorization token.


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
polygon_collection = ee.FeatureCollection(ee.String('users/batuhankavlak4/{YOUR_POLYGON_ASSET}'))
Map.addLayer(polygon_collection, {}, "POLYGON_ASSET")
polygon_feature = ee.Feature(polygon_collection.filter(ee.Filter.eq('FieldId', 324768)).first())
Map.addLayer(polygon_feature, {}, "Field_ID324768")

In [4]:
polygon_feature.getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[33.257198905197725, 37.704659156617566],
    [33.25878403168238, 37.70484003880245],
    [33.259942436345675, 37.70965412548282],
    [33.256345312118064, 37.70941299822742],
    [33.257198905197725, 37.704659156617566]]]},
 'id': '00000000000000000090',
 'properties': {'TarlaId': 324768}}

In [4]:
# Add Earth Engine dataset
s2_col = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(polygon_feature.geometry()) \
    .filterDate('2020-04-01', '2020-11-30') \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12']);

In [7]:
# Image CRS when '.first()' called
ee.Image(s2_col.first().select('B2')).projection().crs().getInfo()

'EPSG:32636'

In [8]:
# Image CRS when '.mean()' called
ee.Image(s2_col.mosaic().select(['B2', 'B3', 'B4'])).projection().crs().getInfo()

'EPSG:4326'

In [12]:
polygon_feature = ee.Image(s2_col.mean().select(['B2'])).sample(**{
    'region': polygon_feature.geometry(),
    'geometries': True,  # if you want points,
    'scale': 10,
    'projection': s2_col.first().select('B2').projection().crs()
});
Map.addLayer(ee.Image(s2_col.mean().select(['B2', 'B3', 'B4'])), {'min': 0, 'max': 2000, 'bands': ['B2', 'B3', 'B4']}, "Base_Image")
Map.addLayer(polygon_feature, {}, "Field_ID324768_Points")

In [ ]:
def set_field_id(id_int):
    '''
    Adds integer as FieldId att to an EE feature object.

    Parameters:
        feature: an EE feature object.
        id_int (int): Id to be added.

    Returns:
        feature(ee feature): an EE feature object. 
    '''
    def wrapper_f(feature):
        base_dict = ee.Dictionary({
            'FieldId': id_int
        })
        feature = feature.set(base_dict)
        return feature
    return wrapper_f


def set_coord_att(feature):
    '''
    Adds coordinates as Coords att to an EE feature object.

    Parameters:
        feature: an EE feature object.

    Returns:
        feature(ee feature): an EE feature object. 
    '''
    coords = feature.geometry().coordinates()
    base_dict = ee.Dictionary({
        'Coords': coords
    })
    feature = feature.set(base_dict)
    return feature


# Generic Function to remove a property from a feature
def remove_property(property_name):
    def wrapper_f(feature):
        properties = feature.propertyNames()
        select_properties = properties.filter(ee.Filter.neq('item', property_name))
        return feature.select(select_properties)
    return wrapper_f

    
polygon_feature.map(remove_property('B2')).getInfo()

In [14]:
# Get PRODUCT_ID in a Python list
collection_list = s2_col.aggregate_array('PRODUCT_ID');
collection_list_py = collection_list.getInfo()

In [15]:
collection_list_py

['S2A_MSIL2A_20200411T083601_N0214_R064_T36SWG_20200411T120406',
 'S2B_MSIL2A_20200413T082559_N0214_R021_T36SWG_20200413T115830',
 'S2B_MSIL2A_20200426T083559_N0214_R064_T36SWG_20200426T123506',
 'S2A_MSIL2A_20200428T082601_N0214_R021_T36SWG_20200428T113331',
 'S2B_MSIL2A_20200513T082559_N0214_R021_T36SWG_20200513T115741',
 'S2B_MSIL2A_20200516T083559_N0214_R064_T36SWG_20200516T123826',
 'S2A_MSIL2A_20200518T082611_N0214_R021_T36SWG_20200518T111906',
 'S2A_MSIL2A_20200521T083611_N0214_R064_T36SWG_20200521T113652',
 'S2A_MSIL2A_20200528T082611_N0214_R021_T36SWG_20200528T113801',
 'S2B_MSIL2A_20200605T083559_N0214_R064_T36SWG_20200605T131458',
 'S2A_MSIL2A_20200607T082611_N0214_R021_T36SWG_20200607T115845',
 'S2A_MSIL2A_20200610T083611_N0214_R064_T36SWG_20200610T112436',
 'S2B_MSIL2A_20200612T082609_N0214_R021_T36SWG_20200612T105411',
 'S2B_MSIL2A_20200612T082609_N0214_R021_T36SWG_20200612T115932',
 'S2B_MSIL2A_20200615T083559_N0214_R064_T36SWG_20200615T123803',
 'S2B_MSIL2A_20200622T082

Export pixel values as a csv

In [ ]:
%%time
# Extract Values in a Loop
for product_name in collection_list_py:
    # Add Earth Engine dataset
    s2_image = ee.Image(s2_col \
                        .filter(ee.Filter.eq('PRODUCT_ID', product_name)) \
                        .first());
    output_dir = os.path.expanduser('./csv/')
    out_csv = os.path.join(output_dir, product_name + '_FieldId324768' + '.csv')
    geemap.extract_values_to_points(polygon_feature, s2_image, out_csv, scale = 10)